In [1]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

# TensorFlow에서 모델 및 레이어 정의

In [2]:
# 모델 생성
class SimpleModule(tf.Module):
    def __init__(self, name = None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5.0, name = "train_me")
        self.a_variable2 = tf.Variable(6.0, name = "train_me2")
        self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
    def __call__(self, x):
        return self.a_variable*x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

In [3]:
# 튜플로 반환
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'train_me2:0' shape=() dtype=float32, numpy=6.0>)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'train_me2:0' shape=() dtype=float32, numpy=6.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


In [4]:
# 레이어 생성
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
        tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

In [5]:
# 모델 생성
class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.        7.6170983]], shape=(1, 2), dtype=float32)


In [6]:
# 모델에 포함되어있는, 하위 레이어 살표보기.
print("Submodules:", my_model.submodules)

Submodules: (<__main__.Dense object at 0x7efe600c9410>, <__main__.Dense object at 0x7efeac788e50>)


In [7]:
for var in my_model.variables:
    print(var, "\n")  

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.51800644, -1.4095653 ,  1.1417068 ],
       [ 0.77221   ,  0.86744255, -0.27350536],
       [ 0.49010974,  1.1222243 ,  0.7961689 ]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[-1.3512522 , -0.85388684],
       [-0.7482452 ,  2.2867675 ],
       [ 0.36775976,  1.8731114 ]], dtype=float32)> 



# 변수 생성 연기하기

In [8]:
class FlexibleDenseModule(tf.Module):
    # Note: No need for `in+features`
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        
        self.is_built = False
        self.out_features = out_features

    def __call__(self, x):
        # Create variables on first call.
        if not self.is_built:
            self.w = tf.Variable(
                tf.random.normal([x.shape[-1], self.out_features]), name='w')
            self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
            self.is_built = True

        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

In [9]:
# Used in a module
class MySequentialModule(tf.Module):       
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = FlexibleDenseModule(out_features=3)
        self.dense_2 = FlexibleDenseModule(out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[2.117933 4.080432]], shape=(1, 2), dtype=float32)


# 가중치 저장

In [10]:
# 데이터 자체와, 메타데이터용 인덱스 파일이라는 두 가지 종류의 파일로 구성됨.
chkp_path = "/content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)
checkpoint.write(chkp_path)

'/content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial'

In [11]:
ls my_checkpoint*

ls: cannot access 'my_checkpoint*': No such file or directory


In [12]:
tf.train.list_variables(chkp_path)

ValueError: ignored

In [ ]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore(chkp_path)

new_model(tf.constant([[2.0, 2.0, 2.0]]))

# 함수 저장

In [ ]:
class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)
    @tf.function
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = MySequentialModule(name="the_model")

In [ ]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

In [ ]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace",
        step=0,
        profiler_outdir=logdir)

In [ ]:
%tensorboard --logdir logs/func

# SavedModel 생성

In [ ]:
tf.saved_model.save(my_model, "the_saved_model")

In [ ]:
# Inspect the in the directory
ls -l the_saved_model

In [ ]:
# The variables/ directory contains a checkpoint of the variables
ls -l the_saved_model/variables

## 마무리 정리

In [ ]:
# 레이어 클래스
class TestDense(tf.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        print("TestDense __init__ callback!")
        self.w = tf.Variable(tf.random.normal([in_features, out_features])) # 가중치
        self.b = tf.Variable(tf.zeros([out_features])) # 바이어스

    def __call__(self, x):
        print("TestDense __call__ callback!")
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

# 모델 (레이어 컨테니어)
class TestModel(tf.Module):
    def __init__(self):
        super().__init__()
        print("TestModel __init__ callback!")
        self.dense_1 = TestDense(in_features=3, out_features=3)
        self.dense_2 = TestDense(in_features=3, out_features=2)
        
    def __call__(self, x):
        print("TestModel __call__ callback!")
        x = self.dense_1(x)
        return self.dense_2(x)

test_model = TestModel()
print(test_model(tf.constant([[2.0, 2.0, 2.0]])))        

# Keras 모델 및 레이어

In [ ]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, in_features, out_features, **kwargs):
        super().__init__(**kwargs)

        self.w = tf.Variable(tf.random.normal([in_features, out_features]))
        self.b = tf.Variable(tf.zeros([out_features]))
    
    def call(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)
simple_layer = MyDense(in_features=3, out_features=3)

In [ ]:
# simple_layer([[2.0, 2.0,2.0]])
simple_layer.variables

In [19]:
class FlexibleDense(tf.keras.layers.Layer):
    def __init__(self, out_features, **kwargs):
        print("FD init")
        super().__init__(**kwargs)
        self.out_features = out_features
    def build(self, input_shape):
        print("FD build")
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]))
        self.b = tf.Variable(tf.zeros([self.out_features]))
    def call(self, inputs):
        print("FD call")
        a = tf.matmul(inputs, self.w) + self.b
        print("@@@@@@@@@",a)
        return a

flexible_dense = FlexibleDense(out_features=3)

FD init


In [20]:
flexible_dense.variables


[]

In [21]:
# Call it, with predictably random results
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

FD build
FD call
@@@@@@@@@ tf.Tensor(
[[-2.6446037  -0.71061254 -2.462738  ]
 [-3.9669058  -1.0659189  -3.6941068 ]], shape=(2, 3), dtype=float32)
Model results: tf.Tensor(
[[-2.6446037  -0.71061254 -2.462738  ]
 [-3.9669058  -1.0659189  -3.6941068 ]], shape=(2, 3), dtype=float32)


In [22]:
try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)

FD call
Failed: In[0] mismatch In[1] shape: 4 vs. 3: [1,4] [3,3] 0 0 [Op:MatMul]


In [23]:
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)
        print("MS init")
        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)
    def call(self, x):
        print("MS call")
        x = self.dense_1(x)
        return self.dense_2(x)
my_sequential_model = MySequentialModel()
print(my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

MS init
FD init
FD init
MS call
FD build
FD call
@@@@@@@@@ tf.Tensor([[-2.2188265 -1.9837594  5.6051936]], shape=(1, 3), dtype=float32)
FD build
FD call
@@@@@@@@@ tf.Tensor([[ 3.4212992 -1.4337698]], shape=(1, 2), dtype=float32)
tf.Tensor([[ 3.4212992 -1.4337698]], shape=(1, 2), dtype=float32)


In [24]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/Variable:0' shape=(3, 3) dtype=float32, numpy=
 array([[-1.1705413 , -0.26449558,  1.1008763 ],
        [-0.06890772, -0.69767   ,  1.3325154 ],
        [ 0.13003571, -0.02971412,  0.3692052 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/Variable:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.7124438 ,  1.1606761 ],
        [ 0.6828128 ,  1.3724616 ],
        [ 0.5700153 ,  0.68939686]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [25]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x7efe061cac90>,
 <__main__.FlexibleDense at 0x7efe061caf10>)

In [28]:
inputs = tf.keras.Input(shape=[3,])
print("input : ",inputs)
x = FlexibleDense(3)(inputs)
print("x1 : ",x)
x = FlexibleDense(2)(x)
print("x2 : ",x)
print()

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

input :  KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_5'), name='input_5', description="created by layer 'input_5'")
FD init
FD build
FD call
@@@@@@@@@ Tensor("flexible_dense_7/add:0", shape=(None, 3), dtype=float32)
x1 :  KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='flexible_dense_7/add:0', description="created by layer 'flexible_dense_7'")
FD init
FD build
FD call
@@@@@@@@@ Tensor("flexible_dense_8/add:0", shape=(None, 2), dtype=float32)
x2 :  KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='flexible_dense_8/add:0', description="created by layer 'flexible_dense_8'")

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
flexible_dense_7 (FlexibleDe (

In [29]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

FD call
@@@@@@@@@ tf.Tensor([[ 0.05498981 -2.2044263  -5.0388746 ]], shape=(1, 3), dtype=float32)
FD call
@@@@@@@@@ tf.Tensor([[ -3.4951618 -19.522823 ]], shape=(1, 2), dtype=float32)


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ -3.4951618, -19.522823 ]], dtype=float32)>